## Desafio 04. Pedidos de usuários

### Bibliotecas

In [1]:
import pandas as pd

# Módulo de conexão com banco
import sys
sys.path.append('../') 
import funcoesconexao as con

# Desativa warnings
import warnings
warnings.filterwarnings('ignore')

### Carregamento dos Dados

#### Via banco de dados — SQL Server

In [2]:
conexao = con.conectar_bd()

----- Conexão com banco de dados estabelecida com sucesso! -----


In [3]:
# Carregando dataframes
df_usuarios = con.carregar_dados(conexao, 'usuarios')
df_pedidos  = con.carregar_dados(conexao, 'pedidos')

# Fechando conexão
con.fechar_conexao_bd(conexao)

In [4]:
# Exibindo dataframes
display(df_usuarios.head())
display(df_pedidos.head())

,id,nome,data_criacao,localizacao_id,email
0,1,João Silva,2019-10-15,1,joao.silva@empresa.com
1,2,Maria Souza,2019-10-15,2,maria.souza@empresa.com
2,3,Carlos Oliveira,2019-10-15,3,carlos.oliveira@empresa.com
3,4,Ana Pereira,2019-10-15,1,ana.pereira@empresa.com
4,5,Pedro Costa,2019-10-15,2,pedro.costa@empresa.com


,id,usuario_id,data_criacao,produto_id,quantidade
0,1,1,2019-01-01,1,10
1,2,3,2019-01-02,2,3
2,3,4,2019-01-02,2,25
3,4,5,2019-01-02,2,14
4,5,5,2019-01-06,3,14


#### Via arquivo .CSV

In [5]:
# Carregando dataframes
df_usuarios = con.carregar_dados_csv('usuarios.csv')
df_pedidos  = con.carregar_dados_csv('pedidos.csv')

In [6]:
# Exibindo dataframes
display(df_usuarios.head())
display(df_pedidos.head())

,id,nome,data_criacao,localizacao_id,email
0,1,João Silva,2019-10-15 00:00:00.000,1,joao.silva@empresa.com
1,2,Maria Souza,2019-10-15 00:00:00.000,2,maria.souza@empresa.com
2,3,Carlos Oliveira,2019-10-15 00:00:00.000,3,carlos.oliveira@empresa.com
3,4,Ana Pereira,2019-10-15 00:00:00.000,1,ana.pereira@empresa.com
4,5,Pedro Costa,2019-10-15 00:00:00.000,2,pedro.costa@empresa.com


,id,usuario_id,data_criacao,produto_id,quantidade
0,1,1,2019-01-01 00:00:00.000,1,10
1,2,3,2019-01-02 00:00:00.000,2,3
2,3,4,2019-01-02 00:00:00.000,2,25
3,4,5,2019-01-02 00:00:00.000,2,14
4,5,5,2019-01-06 00:00:00.000,3,14


### Solução

In [7]:
def pedidos_usuarios(usuarios: pd.DataFrame, pedidos: pd.DataFrame):
    # Convertendo coluna data_criacao para o tipo datetime
    pedidos['data_criacao'] = pd.to_datetime(pedidos['data_criacao'])

    # Filtrando pedidos dos anos 2019 e 2020
    pedidos['ano'] = pedidos['data_criacao'].dt.year
    pedidos_filtrados = pedidos[pedidos['ano'].isin([2019, 2020])]

    # Calcula quantidade de pedidos, agrupando por usuário e ano
    contagem_pedidos = (
        pedidos_filtrados.groupby(['usuario_id', 'ano'])
        .size()
        .unstack(fill_value=0)
        .reset_index()
    )

    # Filtra usuários com mais de 3 pedidos em 2019 e 2020
    contagem_pedidos = contagem_pedidos[
        (contagem_pedidos[2019] > 3) & (contagem_pedidos[2020] > 3)
    ]

    # Mesclando dataframes
    df_merge = usuarios.merge(
        contagem_pedidos, left_on='id', right_on='usuario_id', how='inner'
    )

    # Selecionando e renomeando colunas
    resultado = df_merge[['nome']].rename(columns={'nome': 'usuario_nome'})

    # Ordenando dataframe
    resultado.sort_values(by='usuario_nome', inplace=True)

    # Saída
    return resultado

In [8]:
pedidos_usuarios(df_usuarios, df_pedidos)

,usuario_nome
1,Ana Pereira
0,Carlos Oliveira
